# **Colorir Candles de Tendência**

## ***01.Explicação***

**Condições para tendência:**
- Close acima da média de 20 períodos;
- Média de 20 estar ascendente (atual maior que a máxima dos últimos 5 períodos);

## ***02.Instalação e importação de libs***

In [114]:
!pip install yfinance --quiet

In [119]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

## ***3. Importando dados do ativo via yfinance***

In [121]:
dados = yf.download('PETR4.SA', start='2022-01-01', period='1d', auto_adjust=False)
dados.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,28.540001,29.219999,28.530001,29.090000,17.000307,52704700
2022-01-04,29.160000,29.400000,28.910000,29.200001,17.064590,51739200
2022-01-05,29.190001,29.270000,27.940001,28.070000,16.404211,78459800
2022-01-06,28.290001,28.650000,27.840000,28.049999,16.392527,61163100
2022-01-07,28.110001,28.290001,27.820000,28.180000,16.468498,47507600


## ***4. Criando condições***

Condições para tendência:

* Close acima da média de 20 períodos;
* Média de 20 estar ascendente (atual maior que a dos últimos 5 períodos);

In [122]:
dados['mm_20p'] = dados.Close.rolling(20).mean()
dados['max_mm_ant'] = dados.mm_20p.rolling(5, closed='left').max()
dados['min_mm_ant'] = dados.mm_20p.rolling(5, closed='left').min()
dados['cond_alta'] = (dados.Close > dados.mm_20p) & (dados.mm_20p > dados.max_mm_ant)
dados['cond_baixa'] = (dados.Close < dados.mm_20p) & (dados.mm_20p < dados.min_mm_ant)
dados

,Open,High,Low,Close,Adj Close,Volume,mm_20p,max_mm_ant,min_mm_ant,cond_alta,cond_baixa
Date,,,,,,,,,,,
2022-01-03,28.540001,29.219999,28.530001,29.090000,17.000307,52704700,NaN,NaN,NaN,False,False
2022-01-04,29.160000,29.400000,28.910000,29.200001,17.064590,51739200,NaN,NaN,NaN,False,False
2022-01-05,29.190001,29.270000,27.940001,28.070000,16.404211,78459800,NaN,NaN,NaN,False,False
2022-01-06,28.290001,28.650000,27.840000,28.049999,16.392527,61163100,NaN,NaN,NaN,False,False
2022-01-07,28.110001,28.290001,27.820000,28.180000,16.468498,47507600,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2023-02-22,26.000000,26.110001,25.690001,25.760000,25.760000,38608500,26.0820,26.1335,25.9395,False,False
2023-02-23,25.770000,26.790001,25.770000,26.549999,26.549999,73913400,26.0795,26.1335,25.9985,False,False
2023-02-24,26.670000,26.700001,25.870001,25.900000,25.900000,64409100,26.0275,26.1335,26.0795,False,True


## ***05. Plot***

In [138]:
fig = go.Figure()


fig.add_trace(go.Candlestick(
    x=dados.index,
    open=dados['Open'],
    high=dados['High'],
    low=dados['Low'],
    close=dados['Close'],
    name='sem_tendência', increasing_line_color= 'gray', decreasing_line_color='darkgray'))

fig.add_trace(go.Candlestick(x=dados.index.where(dados['cond_alta']==True),
    open=dados['Open'].where(dados['cond_alta']==True),
    high=dados['High'].where(dados['cond_alta']==True),
    low=dados['Low'].where(dados['cond_alta']==True),
    close=dados['Close'].where(dados['cond_alta']==True),
    increasing_line_color= 'limegreen',
    decreasing_line_color= 'darkgreen',
    name='Tendência_de_alta'))

fig.add_trace(go.Candlestick(x=dados.index.where(dados['cond_baixa']==True),
    open=dados['Open'].where(dados['cond_baixa']==True),
    high=dados['High'].where(dados['cond_baixa']==True),
    low=dados['Low'].where(dados['cond_baixa']==True),
    close=dados['Close'].where(dados['cond_baixa']==True),
    increasing_line_color='lightcoral',
    decreasing_line_color='red',
    name='Tendência_de_baixa'))

fig.add_trace(go.Scatter(name='Sma_20', x=dados.index, y=dados['mm_20p'], marker_color='gray' ))

fig.add_trace(go.Scatter(x=dados.index.where(dados['cond_alta']==True), y=dados['mm_20p'], marker_color='green', showlegend=False)) 
fig.add_trace(go.Scatter(x=dados.index.where(dados['cond_baixa']==True), y=dados['mm_20p'], marker_color='red', showlegend=False)) 

fig.update_layout(template='ggplot2', width=1400, height=800)

fig.show()